<a href="https://colab.research.google.com/github/farahBassoumi/defi-sandwich-attack-detection/blob/main/cleaning_optimization_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
pd.set_option('display.max_columns', None)

# Global memory tracker
initial_memory = None

def log_step(step_number, title):
    print(f"\n{'=' * 20} Step {step_number}: {title} {'=' * 20}\n")

def load_dataset():
    log_step(1, "Load Dataset")
    df = pd.read_csv('bl_sybil_labeled.csv', sep=',', low_memory=False)
    print("Dataset loaded.")
    df.info()
    return df.copy()

def drop_unwanted_columns(df):
    log_step(2, "Drop Unwanted Columns")
    cols_to_drop = [
        'Unnamed: 0', 'timepending',
        *[col for col in df.columns if len(col) > 1000 or ',' in col]
    ]
    print(f"Dropping {len(cols_to_drop)} columns.")
    return df.drop(columns=cols_to_drop, errors='ignore')

def report_and_drop_duplicates(df):
    log_step(3, "Drop Duplicate Rows")
    dup_percent = df.duplicated().mean() * 100
    print(f"Duplicate rows: {dup_percent:.2f}%")
    return df.drop_duplicates()

def clean_column_names(df):
    log_step(4, "Clean Column Names")
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    print("Column names standardized.")
    return df

def show_null_insights(df):
    log_step(5, "Missing Value Report")
    nulls = df.isnull().mean() * 100
    nulls = nulls[nulls > 0].sort_values(ascending=False)
    if not nulls.empty:
        print("Columns with missing values:")
        print(nulls.to_string())
    else:
        print("No missing values found.")

def optimize_dataframe(df):
    log_step(6, "Optimize Memory Usage")
    num_cols = df.select_dtypes(include=['int', 'float']).columns
    with tqdm(total=len(num_cols), desc="Optimizing Numeric Columns", colour='blue') as pbar:
        for col in num_cols:
            dtype_str = str(df[col].dtype)
            if set(df[col].dropna().unique()).issubset({0, 1}):
                df[col] = df[col].astype(bool)
            elif 'int' in dtype_str:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            elif 'float' in dtype_str:
                df[col] = pd.to_numeric(df[col], downcast='float')
            pbar.set_description(f"Optimizing: {col}")
            pbar.update(1)
    return df

def update_pending_times_and_clean(df):
    log_step(7, "Update Pending Times and Clean")
    df['detecttime'] = pd.to_datetime(df['detecttime'], errors='coerce')

    pending_df = df[df['status'] == 'pending']
    confirmed_df = df[df['status'] == 'confirmed']
    common_hashes = set(pending_df['hash']).intersection(set(confirmed_df['hash']))

    pending_df = pending_df[pending_df['hash'].isin(common_hashes)]
    confirmed_df = confirmed_df[confirmed_df['hash'].isin(common_hashes)]
    pending_times = pending_df[['hash', 'detecttime']].rename(columns={'detecttime': 'pending_detecttime'})
    confirmed_df = confirmed_df.merge(pending_times, on='hash', how='left')

    confirmed_df['computed_time_pending'] = (
        confirmed_df['detecttime'] - confirmed_df['pending_detecttime']
    ).dt.total_seconds()

    confirmed_df['time_pending'] = confirmed_df.apply(
        lambda row: row['computed_time_pending']
        if pd.isna(row['time_pending']) else max(row['time_pending'], row['computed_time_pending']),
        axis=1
    )

    confirmed_df = confirmed_df.drop(columns=['pending_detecttime', 'computed_time_pending'])

    df_cleaned = df[df['status'] != 'pending']
    df_cleaned = df_cleaned[~df_cleaned['hash'].isin(common_hashes)]
    df_cleaned = pd.concat([df_cleaned, confirmed_df], ignore_index=True)

    print("Pending-only rows removed and time_pending updated.")
    return df_cleaned

def show_value_percentages(df, top_n=5):
    log_step(8, "Value Distribution per Column")
    with tqdm(total=len(df.columns), desc="Column Distributions", colour='green') as pbar:
        for col in df.columns:
            try:
                value_counts = df[col].value_counts(normalize=True, dropna=False) * 100
                print(f"\nColumn: {col}")
                print(value_counts.head(top_n).round(2).to_string())
                print("-" * 40)
            except Exception as e:
                print(f"Could not process column {col}: {e}")
            pbar.update(1)

def report_memory(df, label=""):
    mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"{label} memory usage: {mem:.2f} MB")
    return mem

def run_pipeline():
    global initial_memory
    total_steps = 10

    with tqdm(total=total_steps, desc="Pipeline Progress", colour='magenta') as pbar:
        print("=" * 60)
        print("STARTING DATA CLEANING & OPTIMIZATION PIPELINE")
        print("=" * 60)

        df = load_dataset()
        pbar.update(1)

        initial_memory = report_memory(df, "Original")
        pbar.update(1)

        df = drop_unwanted_columns(df)
        pbar.update(1)

        df = report_and_drop_duplicates(df)
        pbar.update(1)

        df = clean_column_names(df)
        pbar.update(1)

        show_null_insights(df)
        pbar.update(1)

        df = optimize_dataframe(df)
        pbar.update(1)

        df = update_pending_times_and_clean(df)
        pbar.update(1)

        show_value_percentages(df.copy(), 10)
        pbar.update(1)

        final_memory = report_memory(df, "Optimized")
        pbar.update(1)

        reduction_percent = 100 * (initial_memory - final_memory) / initial_memory
        print(f"\nMemory usage reduced by: {reduction_percent:.2f}%")
        print("\nPipeline completed successfully.")

        return df

# Run the pipeline
cleaned_df = run_pipeline()


Pipeline Progress:   0%|          | 0/10 [00:00<?, ?it/s]

STARTING DATA CLEANING & OPTIMIZATION PIPELINE

==================== Step 1: Load Dataset ====================

Dataset loaded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720649 entries, 0 to 6720648
Data columns (total 30 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Dtype  
---  ------                                                                                                                                                                                                                                              

Optimizing Numeric Columns:   0%|          | 0/14 [00:00<?, ?it/s]


==================== Step 7: Update Pending Times and Clean ====================

Pending-only rows removed and time_pending updated.

==================== Step 8: Value Distribution per Column ====================



Column Distributions:   0%|          | 0/26 [00:00<?, ?it/s]


Column: detecttime
detecttime
NaT                                 0.06
2024-01-15 02:11:48.978000+00:00    0.02
2024-01-13 22:53:48.859000+00:00    0.02
2024-01-13 09:50:01.570000+00:00    0.02
2024-01-15 02:08:37.329000+00:00    0.02
2024-01-18 18:24:37.133000+00:00    0.02
2024-01-15 04:28:50.269000+00:00    0.02
2024-01-15 03:21:48.599000+00:00    0.02
2024-01-15 04:22:36.873000+00:00    0.02
2024-01-15 04:08:49.840000+00:00    0.01
----------------------------------------

Column: hash
hash
0x9408ae4d16bc212201d21b091a87a9608cf2fd2a7715c1189d1986ab6c5d8dae    0.0
0x934f28ef91b84277b853cd98668a3c47631a49fc120d3652d97c85352d3cd097    0.0
0x40d83ee85e1c29c3d39042fb15c4b2433e75782c60897067f67bb3b1ff173129    0.0
0x70726aa541e18eae2784b2743f50780646404187e367041434384fcce99120eb    0.0
0x46ea7caa18915f5a580388ae6de75625d52e0c75efe3bb3e9933480fcc2a5de8    0.0
0x04640f4dad4f4a8ec4d3a1f0bb268644fa2c43fd63752ed01b155af838c26821    0.0
0xa42b2fa408698cd7849faf7b57376707adccbde3223bb9cb9cd55

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df.to_csv('cleaned_df.csv',index=False)